# ASSIGNMENT 1                                                McGill:COMP766-2 - Winter 2021 
Student name (ID)

In [1]:
import numpy as np
import networkx as nx
from utils import tensor_mult

### Problem 1. (5 points)

- __a__: 2 points) Derive the __Markov__ inequality below for a positive discrete random variable
(_Hint:_ rearrange to prove $a P(x \geq a) \leq \mathbb{E}[X]$ and substitute the definition of expectation.)
$$P(x > a) \leq \frac{\mathbb{E}[X]}{a}$$
- __b__: 3 points) Using this inequality prove the following, known as __Chebyshev__ inequality:
$$P(|X - \mathbb{E}[X]| > a) < \frac{Var(X)}{a^2}$$

### Solution
__a__) 

__b__) 

### Problem 2. (5 points)

In showing that factorization in a Markov Network leads to local CIs we used the
following fact. Prove it using the definition of conditional independence:

$$
P \models (X \perp Y \mid Z) \quad \Leftrightarrow \quad P(X=x, Y=y, Z=z) = f(x, z)g(y,z) 
$$


### Solution

- $\Rightarrow$) 2 points 

- $\Leftarrow$) 3 points


### Problem 3. (10 points)

Here, we want to represent the joint probability $P(D,I,G,S,L)$ and answer arbitrary queries such as $P(D,I \mid G=0, L=1)$ for the running example below that we used extensively in the class.
<img src="3_4.png" width="400">
For this, we need to define CPTs. We use the ```networkx``` package to represent a DAG and add CPTs as node attributes. the CPT for a node with $K$ parents is a $K+1$ dimensional array, where the first dimension is for the child node and the order of parents follows their order when the method ```DAG.predecessors(node)``` is called. This is the order in which the corresponding edges are added. 

Your task is to write the body of the function ```Pr()``` that calculates the array of the posterior marginal, given a DAG -- e.g., $P(D, L \mid G= 2, I = 1)$. 
For your implementation you can use the ```tensor_mult``` helper function provided in ```utility.py```. 

You can try implementing this function in three steps:

- calculate the joint PMF
- condition on the evidence (e.g., by multiplying the joint array with appropriate tensor of 0s and 1s.
- marginalize and normalize the final results (normalization might be necessary depending on your implementation of conditioning on the evidence)

There are more efficient ways of calculating the posterior marginals that we study in the __inference__ lectures.

### Solution

In [118]:
# creating the BayesNet
BN = nx.DiGraph()
BN.add_node('D', cpt=np.array([.6,.4]))
BN.add_node('I', cpt=np.array([.7,.3]))

#a 3-dimensional array of shape 3x2x2 representing P(G|I,D).  
#note that the order of parents matters, here we have made sure the order is the same as
#the order returned by BN.predecessors('G')
BN.add_node('G', cpt=np.array([[[.3,.05],[.9,.5]],[[.4,.25],[.08,.3]],[[.3,.7],[.02,.2]]]))
BN.add_node('L', cpt=np.array([[.1,.4,.99],[.9,.6,.01]]))
BN.add_node('S', cpt=np.array([[.95,.2],[.05,.8]]))

# adding edges (note that the order of I,D -> G is important)
BN.add_edge('I','G')
BN.add_edge('D','G')
BN.add_edge('G', 'L')
BN.add_edge('I', 'S')

print("Is this a DAG? {}".format(nx.is_directed_acyclic_graph(BN)))
# we can use topological sort to get a topological ordering of nodes. What is the complexity of this sorting op.?
print(list(nx.topological_sort(BN)))

cpt = nx.get_node_attributes(BN, "cpt")
# print(cpt['G'][0,0])

c1 = tensor_mult(cpt['I'], cpt['S'], [0], [1])
print(c1[0,1])
c2 = tensor_mult(c1, cpt['D'], [], [])
print(c2[1,1])
c3 = tensor_mult(c2, cpt['G'], [0,2], [1,2])
print(c3.shape)
P = tensor_mult(c3, cpt['L'], [3], [1])

print(P[1,1,0,1,0])
a=np.sum(P, axis=(1,3))/0.42   
print(a[0,0,1])
print(np.sum(P, axis=(1,3,4)))
print(np.sum(P, axis=(1,2,3,4)))


def Pr(target, 
       evidence, # a dictionary of observations to conditin on -- eg, {'L':0, 'I':1}
       DAG #DAG containing the CPTs (BN above)
      ):
    
    sorted_BN = list(nx.topological_sort(BN))
    
    # creating the tensor of joint probabilities
    c1 = tensor_mult(cpt['I'], cpt['S'], [0], [1])
    print(c1[0,1])
    c2 = tensor_mult(c1, cpt['D'], [], [])
    print(c2[1,1])
    c3 = tensor_mult(c2, cpt['G'], [0,2], [1,2])
    print(c3.shape)
    P = tensor_mult(c3, cpt['L'], [3], [1])

    #Calculating the margin of non evidences
    evidence_keys = list((evidence.keys()))
    evidence_values = list((evidence.values()))
    margin_denum = P.copy()
    print(evidence_values)
    margin_axises = []
    margin_values = []
    for i,node in enumerate(sorted_BN):
        if node not in evidence_keys:
            margin_axises.append(i)
            margin_denum = margin_denum.sum(axis=i)
        else:
            margin_values.append(evidence[node]) #topological sorting evidence values
    
    
    
    #Calculating the margin of joint prbability of target and evidences
    print(target)
    joint_axises = []
    join_nodes = []
    margin_joint = P.copy()
    for i,node in enumerate(sorted_BN):
        if node not in evidence_keys and node not in target:
            print(i)
            joint_axises.append(i)
            margin_joint = margin_joint.sum(axis=i)
        else:
            join_nodes.append(node)
            
    print(margin_joint.shape)
    
#     slice_axises = []
#     for i,node in enumerate(join_nodes):
#         if node in evidence_keys:
#             slice_axises.append(i)
            
#     for idx in slice_axises:
#         print(margin_joint.shape)
#         margin_joint=margin_joint.take(indices=evidence[join_nodes[i]], axis=idx)
    idx = 0  
    for i,node in enumerate(join_nodes):
        
        print(idx)
        if node in evidence_keys:
            margin_joint = margin_joint.take(indices=evidence[node],axis=idx)
            print(margin_joint.shape)
        else:
            idx +=1
    print(margin_joint)
            
    
    marginal = margin_joint/margin_joint.sum()
    return marginal

evidence = {'L':0,'G':2,'D':1} 
target = ['I']

res = Pr(target,evidence,BN)
print(res)

Is this a DAG? True
['I', 'S', 'D', 'G', 'L']
0.034999999999999996
[0.144 0.096]
(2, 2, 2, 3)
0.004608
0.513
[[0.42 0.28]
 [0.18 0.12]]
[0.7 0.3]
0.034999999999999996
[0.144 0.096]
(2, 2, 2, 3)
[0, 2, 1]
['I']
1
(2, 2, 3, 2)
0
1
(2, 3, 2)
1
(2, 2)
1
(2,)
[0.19404 0.02376]
0.0377982
[0.89090909 0.10909091]


### Problem 4. (10 points)

Your task in this assignment is to implement __D-separation__ algorithm for a DAG representation, similar to what we used in the previous problem. Note that (assuming non-deterministic factors) D-separation does not need access to the CPTs. The following function returns ```True``` if the given CI holds and ```False``` otherwise.

In [5]:
def is_cond_independent(X, #non-empty list of nodes -- e.g., ['I', 'D'] 
                        Y, #non-empty list of nodes. It has no intersection with X -- e.g., ['S']
                        Z, #list of nodes -- e.g., []
                        DAG #networkx DAG -- e.g., BN defined above
                       ):
    # >>>> YOUR CODE HERE >>>>>>>
    
    # >>>> YOUR CODE HERE >>>>>>>

SyntaxError: unexpected EOF while parsing (<ipython-input-5-ae9d1d9cccea>, line 8)

Let us verify that the CIs that we get from D-separation, match the definition of conditional independence using the conditional probabilities that we can numerically calculate using your solution to the problem 3. 
In the following we look at all queries in the form of $P \overset{?}{\models} {D} \perp {S} \mid \mathbf{Z} = \mathbf{0}$, where $\mathbf{Z} \subseteq \{I,G,L\}$ may contain any of the remaining variables. 

In [80]:
domains = {'I':[0,1], 'L':[0,1], 'G':[0,1,2]}
for Z in [{},{'I':0}, {'G':0}, {'L':0}, {'I':0,'G':0}, {'I':0,'L':0}, {'L':0,'G':0}, {'I':0,'G':0,'L':0}]:
    #conditional independence from D-separation
    CI_alg = is_cond_independent(['D'], ['S'], Z.keys(), BN)
    #conditional independence according to conditional probabilities
    CI_num = np.max(np.abs(Pr(['D','S'], Z, BN) - np.outer(Pr(['D'], Z, BN),Pr(['S'], Z, BN)))) < 1e-10
    #they should match
    assert(CI_num == CI_alg)
print("passed the minimal test")

NameError: name 'is_cond_independent' is not defined

In [81]:
a = np.ones([2,2,3,5,4])
b = np.random.rand(7,4,3,2)
c = tensor_mult(a, b, [1,4], [3,1])
c.shape # [2,2,3,5,4,7,3] 

print(a,b,c)

[[[[[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]

   [[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]

   [[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]]


  [[[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]

   [[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]

   [[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]]]



 [[[[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]

   [[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]

   [[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]]


  [[[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]]

   [[1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
    [1. 1. 1. 1.]
